In [33]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import threading
import time
import reverse_geocoder as rg

shootings = pd.read_csv('shootings.csv')
city_loc_df = pd.read_csv('city_loc2.csv')
city_and_state_to_geo_code = {city: [lon, lat] for city, lon, lat in city_loc_df.values}

In [34]:
city_and_state_to_geo_code

{'New York,NY': [40.6943, -73.9249],
 'Los Angeles,CA': [34.1139, -118.4068],
 'Chicago,IL': [41.8373, -87.6862],
 'Miami,FL': [25.7839, -80.2102],
 'Dallas,TX': [32.7936, -96.7662],
 'Philadelphia,PA': [40.0077, -75.1339],
 'Houston,TX': [29.7863, -95.3889],
 'Atlanta,GA': [33.7627, -84.4224],
 'Washington,DC': [38.9047, -77.0163],
 'Boston,MA': [42.3188, -71.0846],
 'Phoenix,AZ': [33.5722, -112.0891],
 'Seattle,WA': [47.6211, -122.3244],
 'San Francisco,CA': [37.7562, -122.443],
 'Detroit,MI': [42.3834, -83.1024],
 'San Diego,CA': [32.8312, -117.1225],
 'Minneapolis,MN': [44.9635, -93.2678],
 'Tampa,FL': [27.9942, -82.4451],
 'Denver,CO': [39.7621, -104.8759],
 'Brooklyn,NY': [40.6501, -73.9496],
 'Queens,NY': [40.7498, -73.7976],
 'Riverside,CA': [33.9381, -117.3948],
 'Baltimore,MD': [39.3051, -76.6144],
 'Las Vegas,NV': [36.2333, -115.2654],
 'Portland,OR': [45.5372, -122.65],
 'San Antonio,TX': [29.4658, -98.5253],
 'St. Louis,MO': [38.6358, -90.2451],
 'Sacramento,CA': [38.5667,

In [8]:
def format(city, state):
    return '%s,%s' % (city, state)

def parse_geocode(s):
    return [float(i) for i in s.split(', ')]

def get_geocode(s):
    geolocator = Nominatim(user_agent="hw3")
    loc = geolocator.geocode(s + ',US')
    return (loc.latitude, loc.longitude)

In [19]:
city_loc_values = []
coordinates = []
for city, s in city_loc_df.values:
    city_loc_values.append([city, parse_geocode(s)])
    coordinates.append(tuple(parse_geocode(s)))
state_names = ["Alaska", "Alabama", "Arkansas", "Arizona", "California", "Colorado", "Connecticut", "Washington, D.C.","Delaware", "Florida","Georgia", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina","South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]
res = rg.search(tuple(coordinates))

i = 0

for value, info in zip(city_loc_values, res):
    if info['admin1'] not in state_names:
        lat, lon = get_geocode(value[0])
        city_loc_values[i][1] = [lat, lon]
    i += 1

In [22]:
i = 0
coordinates = []
for _, (lat, lon) in city_loc_values:
    coordinates.append((lat, lon))
res = rg.search(tuple(coordinates))

for value, info in zip(city_loc_values, res):
    if info['admin1'] not in state_names:
        lat, lon = get_geocode(value[0])
        print(value, lat, lon)
        city_loc_values[i][1] = [lat, lon]
    i += 1

['Chapeno,TX', [26.53466625, -99.14969576259185]] 26.53466625 -99.14969576259185
['San Ysidro,CA', [32.5520013, -117.0430813]] 32.5520013 -117.0430813


In [27]:
city_loc_np_arr = np.array([[i, j[0], j[1]] for i, j in city_loc_values])

city_loc_np_df = pd.DataFrame(data=city_loc_np_arr, columns=['city', 'latitude', 'longitude'])

In [31]:
city_loc_np_df.to_csv('city_loc2.csv', index=False)

In [3]:
city_state = shootings[['city', 'state']].values
longitude = []
latitude = []
used = set()
for city, state in city_state:
    format_str = format(city, state)
    lat, long = city_and_state_to_geo_code[format_str].split(',')
    used.add(city_and_state_to_geo_code[format_str])
    longitude.append(float(long))
    latitude.append(float(lat))

In [4]:
shootings.insert(len(shootings.columns), "longitude", longitude)
shootings.insert(len(shootings.columns), "latitude", latitude)
shootings.to_csv('shootings_with_geocode.csv')